In [1]:
from datasets import *
from train import *
from torchvision import datasets, transforms, models

training

In [2]:
teacher = models.efficientnet_b0(weights='DEFAULT').to('cuda').eval()
student = models.shufflenet_v2_x0_5(weights='DEFAULT').to('cuda').eval()

In [3]:
with torch.no_grad():
    # first reinitialize the layer before classification to match the teacher feature dimensions
    student.conv5[0] = torch.nn.Conv2d(student.conv5[0].in_channels,teacher.classifier[1].in_features,kernel_size=(1, 1), stride=(1, 1), bias=False)
    student.conv5[1] = torch.nn.BatchNorm2d(teacher.classifier[1].in_features,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)

    # next create a new fc layer to match the teacher dimension
    student.fc = torch.nn.Linear(teacher.classifier[1].in_features,teacher.classifier[1].out_features)

    # finally copy the teacher fc parameters
    student.fc.weight[:,:] = teacher.classifier[1].weight[:,:]
    student.fc.bias[:] = teacher.classifier[1].bias[:]

    # freeze the classification layer
    for param in student.fc.parameters():
        param.requires_grad = False

    student.to('cuda')

In [4]:
train_loader, val_loader = load_imagenet(64,12345)
train(student,train_loader,val_loader,'cuda',lr=0.004,epochs=100,log_name="sn_frozen_headval")

2023-04-27 13:33:35.626753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-27 13:33:35.759191: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 13:33:36.365110: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Train Epoch: 0 [0/1281167 (0%)] train loss: 7.200, lr: 0.00400000
Train Epoch: 0 [51200/1281167 (4%)] train loss: 4.650, lr: 0.00400000
Train Epoch: 0 [102400/1281167 (8%)] train loss: 4.277, lr: 0.00398000
Train Epoch: 0 [153600/1281167 (12%)] train loss: 3.504, lr: 0.00396010
Train Epoch: 0 [204800/1281167 (16%)] train loss: 3.465, lr: 0.00394030
Train Epoch: 0 [256000/1281167 (20%)] train loss: 3.473, lr: 0.00392060
Train Epoch: 0 [307200/1281167 (24%)] train loss: 2.986, lr: 0.00390100
Train Epoch: 0 [358400/1281167 (28%)] train loss: 3.011, lr: 0.00388149
Train Epoch: 0 [409600/1281167 (32%)] train loss: 2.918, lr: 0.00386208
Train Epoch: 0 [460800/1281167 (36%)] train loss: 3.220, lr: 0.00384277
Train Epoch: 0 [512000/1281167 (40%)] train loss: 3.184, lr: 0.00382356
Train Epoch: 0 [563200/1281167 (44%)] train loss: 2.823, lr: 0.00380444
Train Epoch: 0 [614400/1281167 (48%)] train loss: 2.750, lr: 0.00378542
Train Epoch: 0 [665600/1281167 (52%)] train loss: 2.800, lr: 0.00376649
T

100%|██████████| 782/782 [01:10<00:00, 11.05it/s]

Train Epoch: 0 [300270/1281167 (100%)] train loss: 3.235, val loss: 2.713, val acc: 0.404, top5: 0.669, lr: 0.00352888
==================== best validation accuracy ====================
epoch: 0, val accuracy: 0.40392


Train Epoch: 1 [0/1281167 (0%)] train loss: 2.234, lr: 0.00352888
Train Epoch: 1 [51200/1281167 (4%)] train loss: 3.031, lr: 0.00352888
Train Epoch: 1 [102400/1281167 (8%)] train loss: 2.628, lr: 0.00175562
Train Epoch: 1 [153600/1281167 (12%)] train loss: 2.742, lr: 0.00174684
Train Epoch: 1 [204800/1281167 (16%)] train loss: 2.671, lr: 0.00173811
Train Epoch: 1 [256000/1281167 (20%)] train loss: 3.132, lr: 0.00172942
Train Epoch: 1 [307200/1281167 (24%)] train loss: 2.300, lr: 0.00172077
Train Epoch: 1 [358400/1281167 (28%)] train loss: 3.214, lr: 0.00171216
Train Epoch: 1 [409600/1281167 (32%)] train loss: 2.964, lr: 0.00170360
Train Epoch: 1 [460800/1281167 (36%)] train loss: 2.468, lr: 0.00169509
Train Epoch: 1 [512000/1281167 (40%)] train loss: 2.569, lr: 0.00168661
Train Epoch: 1 [563200/1281167 (44%)] train loss: 2.460, lr: 0.00167818
Train Epoch: 1 [614400/1281167 (48%)] train loss: 2.506, lr: 0.00166979
Train Epoch: 1 [665600/1281167 (52%)] train loss: 2.620, lr: 0.00166144
T

...

Train Epoch: 8 [0/1281167 (0%)] train loss: 2.484, lr: 0.00001147
Train Epoch: 8 [51200/1281167 (4%)] train loss: 1.477, lr: 0.00001147
Train Epoch: 8 [102400/1281167 (8%)] train loss: 1.885, lr: 0.00000571
Train Epoch: 8 [153600/1281167 (12%)] train loss: 1.619, lr: 0.00000568
Train Epoch: 8 [204800/1281167 (16%)] train loss: 1.880, lr: 0.00000565
Train Epoch: 8 [256000/1281167 (20%)] train loss: 2.271, lr: 0.00000562
Train Epoch: 8 [307200/1281167 (24%)] train loss: 2.570, lr: 0.00000559
Train Epoch: 8 [358400/1281167 (28%)] train loss: 2.049, lr: 0.00000556
Train Epoch: 8 [409600/1281167 (32%)] train loss: 1.955, lr: 0.00000554
Train Epoch: 8 [460800/1281167 (36%)] train loss: 1.774, lr: 0.00000551
Train Epoch: 8 [512000/1281167 (40%)] train loss: 2.018, lr: 0.00000548
Train Epoch: 8 [563200/1281167 (44%)] train loss: 1.558, lr: 0.00000545
Train Epoch: 8 [614400/1281167 (48%)] train loss: 1.775, lr: 0.00000543
Train Epoch: 8 [665600/1281167 (52%)] train loss: 1.770, lr: 0.00000540
T

100%|██████████| 782/782 [01:12<00:00, 10.86it/s]

Train Epoch: 8 [300270/1281167 (100%)] train loss: 2.613, val loss: 2.185, val acc: 0.511, top5: 0.753, lr: 0.00000506


Train Epoch: 9 [0/1281167 (0%)] train loss: 1.797, lr: 0.00000506
Train Epoch: 9 [51200/1281167 (4%)] train loss: 1.555, lr: 0.00000506
Train Epoch: 9 [102400/1281167 (8%)] train loss: 1.882, lr: 0.00000252
Train Epoch: 9 [153600/1281167 (12%)] train loss: 1.766, lr: 0.00000250
Train Epoch: 9 [204800/1281167 (16%)] train loss: 1.291, lr: 0.00000249
Train Epoch: 9 [256000/1281167 (20%)] train loss: 1.908, lr: 0.00000248
Train Epoch: 9 [307200/1281167 (24%)] train loss: 1.706, lr: 0.00000247
Train Epoch: 9 [358400/1281167 (28%)] train loss: 2.092, lr: 0.00000245
Train Epoch: 9 [409600/1281167 (32%)] train loss: 1.375, lr: 0.00000244
Train Epoch: 9 [460800/1281167 (36%)] train loss: 1.972, lr: 0.00000243
Train Epoch: 9 [512000/1281167 (40%)] train loss: 1.638, lr: 0.00000242
Train Epoch: 9 [563200/1281167 (44%)] train loss: 1.984, lr: 0.00000241
Train Epoch: 9 [614400/1281167 (48%)] train loss: 1.637, lr: 0.00000239
Train Epoch: 9 [665600/1281167 (52%)] train loss: 1.619, lr: 0.00000238
T

KeyboardInterrupt: 

testing

In [13]:
test_loader = load_imagenet(128,1234,False)

In [12]:
torch.load("best_batch_i288264sn_frozen_head_long1682578354.1065493.pth")['lr']

[5.142319898206824e-06]

In [11]:
student = models.shufflenet_v2_x0_5(weights='DEFAULT').to('cuda')
with torch.no_grad():
    # first reinitialize the layer before classification to match the teacher feature dimensions
    student.conv5[0] = torch.nn.Conv2d(student.conv5[0].in_channels,teacher.classifier[1].in_features,kernel_size=(1, 1), stride=(1, 1), bias=False)
    student.conv5[1] = torch.nn.BatchNorm2d(teacher.classifier[1].in_features,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True)

    # next create a new fc layer to match the teacher dimension
    student.fc = torch.nn.Linear(teacher.classifier[1].in_features,teacher.classifier[1].out_features)

    # freeze the classification layer
    for param in student.fc.parameters():
        param.requires_grad = False
student.load_state_dict(torch.load("best_batch_i288264sn_frozen_head_long1682578354.1065493.pth")['model_state_dict'])
student.to('cuda')
student.train()

ShuffleNetV2(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (stage2): Sequential(
    (0): InvertedResidual(
      (branch1): Sequential(
        (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=24, bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (3): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (4): ReLU(inplace=True)
      )
      (branch2): Sequential(
        (0): Conv2d(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_

In [14]:
acc = validate(student,test_loader,'cuda')

100%|██████████| 391/391 [04:39<00:00,  1.40it/s]


In [15]:
acc

(0.50772, 2.187045480436681, 0.75248)